## License
```
Visualize scientific satellites orbits.
Copyright (C) 2019+  Benjamin Winkel (bwinkel@mpifr.de)

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

Note: parts of this software were adapted from Cees Bassa (ASTRON);
      see https://github.com/cbassa/satellite_analysis
```

In [1]:
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
# for animations, you may need to install "ffmpeg" and/or "imagemagick"
from matplotlib import animation, rc

import cysgp4

rc('animation', html='html5')

If you plan to make big movies (i.e., large file sizes), you may need to start Jupyter with the command line option ``--NotebookApp.iopub_data_rate_limit=1.0e10`` and do the following:

In [2]:
# matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

In [10]:
science_tle_text = cysgp4.get_example_tles()
my_tles = cysgp4.tles_from_text(science_tle_text)

In [4]:
my_tles[:10], len(my_tles)

([<PyTle: AKEBONO (EXOS-D)        >,
  <PyTle: HST                     >,
  <PyTle: POLAR                   >,
  <PyTle: SWAS                    >,
  <PyTle: ORSTED                  >,
  <PyTle: CXO                     >,
  <PyTle: XMM-NEWTON              >,
  <PyTle: TERRA                   >,
  <PyTle: CLUSTER II-FM7 (SAMBA)  >,
  <PyTle: CLUSTER II-FM6 (SALSA)  >],
 66)

Alternatively, one could also fetch all active satellites from [Celestrack](http://celestrak.com):

In [5]:
# import requests

# ctrak_latest = requests.get('http://celestrak.com/NORAD/elements/active.txt')
# active_tles = cysgp4.tles_from_text(ctrak_latest.text)

To allow numpy broadcasting further below, we need to convert to an array:

In [11]:
my_tles = np.array(my_tles)

As the TLEs are quickly outdated (in which case, the orbit calculations would have large errorbars), we should base our visualization on a date and time, which is close to the epoch of the TLEs.

In [12]:
epoch_dt = my_tles[0].epoch
epoch_dt, epoch_dt.mjd

(<PyDateTime: 2019-11-17 11:58:52.232160 UTC>, 58804.49921565)

We want to visualize the orbits for a short time frame of 10 minutes, in steps of 10 seconds (aka 60 frames).

In [14]:
start_mjd = 58804.5
td = np.arange(0, 600, 10) / 86400.  # 1 d in steps of 10 s
mjds = start_mjd + td

The final ingredient is one or more observer (for the topocentric coordinates), lets define two:

In [15]:
# Effelsberg 100-m radio telescope
effbg_observer = cysgp4.PyObserver(6.88375, 50.525, 0.366)
# Parkes telescope ("The Dish")
parkes_observer = cysgp4.PyObserver(148.25738, -32.9933, 414.8)
observers = np.array([effbg_observer, parkes_observer])

In [16]:
result = cysgp4.propagate_many(
    mjds[np.newaxis, np.newaxis, :],
    my_tles[:, np.newaxis, np.newaxis],
    observers[np.newaxis, :, np.newaxis]
    )

``result`` is a Python dictionary, with the following entries:

In [17]:
print(list(result.keys()))

['eci_pos', 'eci_vel', 'geo', 'topo']


Each entry is an array containing the coordinates of the frame (on the last axis). The broadcasted input arrays form the first axes of the output:

In [20]:
eci_pos = result['eci_pos']
topo_pos = result['topo']
eci_pos.shape, topo_pos.shape

((66, 2, 60, 3), (66, 2, 60, 4))

Unpacking this is a bit ugly, because the coordinate tuples are in the last axis.

In [29]:
eci_pos_x, eci_pos_y, eci_pos_z = (eci_pos[..., i] for i in range(3))
topo_pos_az, topo_pos_el, topo_pos_dist, _ = (topo_pos[..., i] for i in range(4))
topo_pos_az = (topo_pos_az + 180.) % 360. - 180.

In [34]:
my_time = cysgp4.PyDateTime()
my_time.mjd = mjds[0]
plim1, plim2 = 8000, 46000

fig = plt.figure(figsize=(12.8, 7.2))
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')
axes = [ax1, ax2]

for ax, plim in [(ax1, plim1), (ax2, plim2)]:
    ax.view_init(azim=60, elev=30)
    ax.set_xlim((-plim, plim))
    ax.set_ylim((-plim, plim))
    ax.set_zlim((-plim, plim))
    ax.set_aspect('equal')
    ax.set_xlabel('x [km]')
    ax.set_ylabel('y [km]')
    ax.set_zlabel('z [km]')

rads = np.sqrt(eci_pos_x[:, 0, 0] ** 2 + eci_pos_y[:, 0, 0] ** 2 + eci_pos_z[:, 0, 0] ** 2)
# need a copy for clipping in each subplot
eci_pos1 = np.array([eci_pos_x, eci_pos_y, eci_pos_z]).copy()
eci_pos2 = np.array([eci_pos_x, eci_pos_y, eci_pos_z]).copy()

m1 = (
    np.all(np.abs(eci_pos1[0]) < plim1, axis=2) & 
    np.all(np.abs(eci_pos1[1]) < plim1, axis=2) & 
    np.all(np.abs(eci_pos1[2]) < plim1, axis=2)
    )
m2 = (
    np.all(np.abs(eci_pos2[0]) < plim2, axis=2) & 
    np.all(np.abs(eci_pos2[1]) < plim2, axis=2) & 
    np.all(np.abs(eci_pos2[2]) < plim2, axis=2)
    )
eci_pos1 = eci_pos1[: ,m1].reshape(
    (eci_pos1.shape[0], -1, eci_pos1.shape[2], eci_pos1.shape[3])
    )

eci_pos2 = eci_pos2[: ,m2].reshape(
    (eci_pos2.shape[0], -1, eci_pos2.shape[2], eci_pos2.shape[3])
    )
rads1, rads2 = rads.copy(), rads.copy()
rads1 = rads1[m1[:, 0]]
rads2 = rads2[m2[:, 0]]


all_points = [
    ax.scatter(
        ep[0, :, 0, 0], ep[1, :, 0, 0], ep[2, :, 0, 0],
        c=rds, s=1, vmin=6400, vmax=plim, marker='o'
        )
    for ax, plim, ep, rds in [(ax1, plim1, eci_pos1, rads1), (ax2, plim2, eci_pos2, rads2)]
    ]
all_titles = [
    ax.set_title('{:%y/%m/%d %H:%M:%S}'.format(my_time.datetime), loc='center', fontsize=20)
    for ax in axes
    ]

def animate(i):
    all_points[0]._offsets3d = (
        eci_pos1[0, :, 0, i], eci_pos1[1, :, 0, i], eci_pos1[2, :, 0, i]
        )
    all_points[1]._offsets3d = (
        eci_pos2[0, :, 0, i], eci_pos2[1, :, 0, i], eci_pos2[2, :, 0, i]
        )
    for title in all_titles:
        my_time.mjd = mjds[i]
        title.set_text('{:%y/%m/%d %H:%M:%S}'.format(my_time.datetime))
    return (*all_points, *all_titles)


def init():
    return animate(0)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(
    fig, animate, init_func=init, frames=eci_pos_x.shape[2], interval=20, blit=True
    # fig, animate, init_func=init, frames=10, interval=20, blit=True
    )

# this takes a while!
plt.close(anim._fig)
# anim

In [35]:
FFwriter = animation.FFMpegWriter(
    fps=30, bitrate=8000,
    extra_args=['-vcodec', 'libx264'],
    )
anim.save('active_satellites_eci_4k.mp4', writer=FFwriter, dpi=300)

In [30]:
my_time = cysgp4.PyDateTime()
my_time.mjd = mjds[0]
vmin, vmax = np.log10(100), np.log10(50000)

fig = plt.figure(figsize=(12.8, 7.2))
ax1 = fig.add_axes((0.1, 0.5, 0.8, 0.35))
ax2 = fig.add_axes((0.1, 0.1, 0.8, 0.35))
cax = fig.add_axes((0.91, 0.2, 0.02, 0.5))
ax2.set_xlabel('Azimuth [deg]')
ax1.set_ylabel('Elevation [deg]')
for ax in [ax1, ax2]:
    ax.set_xlim((-180, 180))
    ax.set_ylim((0, 90))
    ax.set_xticks(range(-150, 180, 30))
    ax.set_yticks(range(0, 91, 30))
    ax.set_aspect('equal')
    ax.grid()

points1 = ax1.scatter(
    topo_pos_az[:, 1, 0], topo_pos_el[:, 1, 0],
    c=np.log10(topo_pos_dist[:, 1, 0]),
    cmap='viridis', vmin=vmin, vmax=vmax,
    )
points2 = ax2.scatter(
    topo_pos_az[:, 0, 0], topo_pos_el[:, 0, 0],
    c=np.log10(topo_pos_dist[:, 0, 0]),
    cmap='viridis', vmin=vmin, vmax=vmax,
    )
cbar = fig.colorbar(points1, cax=cax)
cbar.set_label('Distance (km)')
cbar.set_ticks([2, 3, 4])
cbar.set_ticklabels([100, 1000, 10000])

ax1.text(-170, 75, 'Parkes 64-m', fontsize=16)
ax2.text(-170, 75, 'Effelsberg 100-m', fontsize=16)
title = ax1.text(
    174, 75, '{:%y/%m/%d %H:%M:%S}'.format(my_time.datetime),
    fontsize=15, ha='right'
    )

def init():
    points1.set_offsets(np.column_stack([topo_pos_az[:, 1, 0], topo_pos_el[:, 1, 0]]))
    points1.set_array(np.log10(topo_pos_dist[:, 1, 0]))
    points2.set_offsets(np.column_stack([topo_pos_az[:, 0, 0], topo_pos_el[:, 0, 0]]))
    points2.set_array(np.log10(topo_pos_dist[:, 0, 0]))
    my_time.mjd = mjds[0]
    title.set_text('{:%y/%m/%d %H:%M:%S}'.format(my_time.datetime))
    return points1, points2, title

def animate(i):
    points1.set_offsets(np.column_stack([topo_pos_az[:, 1, i], topo_pos_el[:, 1, i]]))
    points1.set_array(np.log10(topo_pos_dist[:, 1, i]))
    points2.set_offsets(np.column_stack([topo_pos_az[:, 0, i], topo_pos_el[:, 0, i]]))
    points2.set_array(np.log10(topo_pos_dist[:, 0, i]))
    my_time.mjd = mjds[i]
    title.set_text('{:%y/%m/%d %H:%M:%S}'.format(my_time.datetime))
    return points1, points2, title

anim = animation.FuncAnimation(
    fig, animate, init_func=init, frames=topo_pos_az.shape[2], interval=20, blit=True
    # fig, animate, init_func=init, frames=10, interval=20, blit=True,
    )

# this takes a while!
plt.close(anim._fig)
# anim

In [33]:
FFwriter = animation.FFMpegWriter(
    fps=30, bitrate=8000,
    extra_args=['-vcodec', 'libx264'],
    )
anim.save('active_satellites_horizon_4k.mp4', writer=FFwriter, dpi=300)

In [29]:
3840 / 300, 2160 / 300

(12.8, 7.2)

In [ ]:
plt.close()
plt.plot(topo_pos_az[12, 0, :], topo_pos_el[12, 0, :], 'x')
plt.show()

In [ ]:
my_sat = cysgp4.Satellite(my_tles[12], observer=observers[0])
az, el = [], []
for mjd in mjds:
    my_sat.mjd = mjd
    tp = my_sat.topo_pos()
    az.append(tp.az)
    el.append(tp.el)

In [ ]:
plt.close()
plt.plot(az, el, 'x')
plt.show()